In [2]:
import dlib
import cv2
import numpy as np
from skimage import io, transform
import matplotlib.pyplot as plt
import pandas as pd
import glob
import openface
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder
%matplotlib inline


In [3]:
characters = ['sakib','manash','sourav']
face_detector = dlib.get_frontal_face_detector()
face_recognition_model = './data/model/dlib_face_recognition_resnet_model_v1.dat'
face_pose_predictor = dlib.shape_predictor('./data/model/shape_predictor_68_face_landmarks.dat')
face_encoder = dlib.face_recognition_model_v1(face_recognition_model)
df = pd.DataFrame()

for char in characters:
    l = []
    for filename in glob.glob('./data/train/%s/*' % char):
        image = cv2.imread(filename)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        detected_faces = face_detector(image, 1)
        face_detect = detected_faces[0]
        pose_landmarks = face_pose_predictor(image, face_detect)
        face_encoding = np.array(face_encoder.compute_face_descriptor(image, pose_landmarks, 1))
        l.append(np.append(face_encoding, [char]))
    
    temp = pd.DataFrame(np.array(l))
    df = pd.concat([df, temp])

df.reset_index(drop=True, inplace=True)
    

In [4]:
le = LabelEncoder()
y = le.fit_transform(df[128])
print("Training for {} classes.".format(len(le.classes_)))
X = df.drop(128, axis=1)
print('{} columns for X'.format(len(X.columns)))


Training for 3 classes.
128 columns for X


In [5]:
clf = SVC(C=1, kernel='linear', probability=True)
clf.fit(X, y)


SVC(C=1, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='linear', max_iter=-1, probability=True, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [13]:
font = cv2.FONT_HERSHEY_SIMPLEX
image = cv2.imread('./data/test/6.jpg')
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
img = np.copy(image)
detected_faces = face_detector(image, 1)
for face_detect in detected_faces:
    cv2.rectangle(img, (face_detect.left(), face_detect.top()), (face_detect.right(), face_detect.bottom()), (255, 0, 0), 2)
    pose_landmarks = face_pose_predictor(image, face_detect)
    face_encoding = np.array(face_encoder.compute_face_descriptor(image, pose_landmarks, 1))
    p = clf.predict_proba(face_encoding.reshape(1, 128))
    print(p)
    if np.max(p) > 0.8:
        print(p)
        y_pred = le.inverse_transform(np.argmax(p))
        cv2.putText(img, y_pred, (face_detect.left(), face_detect.top()-5), font, 0.9, (255, 0, 0))
plt.figure(figsize=(20,10))
plt.imshow(img)


[[0.01536182 0.04858833 0.93604985]]
[[0.01536182 0.04858833 0.93604985]]


ValueError: bad input shape ()